In [94]:
import numpy as np
import pandas as pd
import xarray as xr
from scipy import ndimage as ndi

In [95]:
proj_path = "/vols/satellite/home/lenk/proj"

In [4]:
haci_cases_20100524 = pd.read_csv("{}/2019-09_bell/haci_cases_20100524_filtered.csv".format(proj_path))

In [97]:
with xr.open_dataset("{}/2019-09_bell/objektmaske_20100524_eu.nc".format(proj_path),decode_cf=False) as f:
    objektmaske = f.object_mask.data

In [99]:
def time2index(time_string,time_intervall=5):
    hours = (int(time_string.split(":")[0]) * 60) / time_intervall
    minutes = int(time_string.split(":")[1])
    
    return int(hours + minutes)

In [118]:
object_ids = []

for i, row in haci_cases_20100524.iterrows():
    tidx = time2index(row.time)
    
    object_id = objektmaske[tidx][row.msg_r_corr,row.msg_c_corr]
    
    if object_id == 0:
        object_cutout = objektmaske[tidx][row.msg_r_corr-10:row.msg_r_corr+10,
                                          row.msg_c_corr-10:row.msg_c_corr+10]
        
        if np.all(object_cutout) == 0:
            object_ids.append(np.nan)
        else:
            obj_ids = np.unique(object_cutout)[1:]
            centroids = ndi.measurements.center_of_mass(object_cutout)


            if len(centroids) < 3:
                object_ids.append(obj_ids[0])
            else:
                distances = []
                for c in centroids:
                    d = np.sqrt((5-c[0])**2 + (5 - c[1])**2)

                object_ids.append(obj_ids[np.argmin(distances)])
    else:
        object_ids.append(object_id)

In [121]:
haci_cases_20100524 = haci_cases_20100524.assign(object_id = object_ids)

In [122]:
haci_cases_20100524.to_csv("{}/2019-09_bell/haci_cases_20100524_filtered.csv".format(proj_path),index=False)